In [1]:
import sys
import numpy as np
import pandas as pd
import warnings
import plotly.graph_objects as go
import plotly.figure_factory as ff
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
sys.path.insert(0,'/media/csivsw/crossOS/playground/friends_of_tracking/src/friends_of_tracking/LaurieOnTracking')
sys.path.insert(1,'/media/csivsw/crossOS/playground/friends_of_tracking/dashing/')

%matplotlib inline

In [2]:
import Metrica_IO as mio
import Metrica_Viz as mviz
import Metrica_Velocities as mvel
import Metrica_PitchControl as mpc

In [3]:
from helper import plot_pitch_slider


func:'populate_LIV_tracking_dataframes' took: 0.1258 sec


# Read data

In [5]:
data_attack = pd.read_csv('../src/friends_of_tracking/Last-Row/datasets/positional_data/GOALVID19_tracking_attack.csv',index_col=[0,1])
data_defence = pd.read_csv('../src/friends_of_tracking/Last-Row/datasets/positional_data/GOALVID19_tracking_defence.csv',index_col=[0,1])
data_attack.shape, data_defence.shape

((3549, 113), (3549, 108))

In [6]:
scores = ['BAY 0 [1] LIV',
'BOU 0 [3] LIV',
'FUL 0 [1] LIV',
'GNK 0 [3] LIV',
'LEI 0 [3] LIV',
'LIV [1] 0 EVE',
'LIV [1] 0 WAT',
'LIV [1] 0 WLV',
'LIV [2] 0 EVE',
'LIV [2] 0 MCI',
'LIV [2] 0 POR',
'LIV [2] 0 RBS',
'LIV [2] 1 CHE',
'LIV [2] 1 NEW',
'LIV [3] 0 BOU',
'LIV [3] 0 NOR',
'LIV [4] 0 FCB',
'POR 0 [2] LIV',
'SOU 1 [2] LIV']
for play, score in list(zip(data_attack.index.get_level_values(0).unique(),scores)):
    print (play,score)
    data_attack.loc[play,'score'] = score
    data_defence.loc[play,'score'] = score    

Bayern 0 - [1] Liverpool BAY 0 [1] LIV
Bournemouth 0 - 3 Liverpool BOU 0 [3] LIV
Fulham 0 - [1] Liverpool FUL 0 [1] LIV
Genk 0 - [3] Liverpool GNK 0 [3] LIV
Leicester 0 - [3] Liverpool LEI 0 [3] LIV
Liverpool [1] - 0 Everton LIV [1] 0 EVE
Liverpool [1] - 0 Watford LIV [1] 0 WAT
Liverpool [1] - 0 Wolves LIV [1] 0 WLV
Liverpool [2] - 0 Everton LIV [2] 0 EVE
Liverpool [2] - 0 Man City LIV [2] 0 MCI
Liverpool [2] - 0 Porto LIV [2] 0 POR
Liverpool [2] - 0 Salzburg LIV [2] 0 RBS
Liverpool [2] - 1 Chelsea LIV [2] 1 CHE
Liverpool [2] - 1 Newcastle LIV [2] 1 NEW
Liverpool [3] - 0 Bournemouth LIV [3] 0 BOU
Liverpool [3] - 0 Norwich LIV [3] 0 NOR
Liverpool [4] - 0 Barcelona LIV [4] 0 FCB
Porto 0 - [2] Liverpool POR 0 [2] LIV
Southampton 1 - [2] Liverpool SOU 1 [2] LIV


In [7]:
play = 'Bayern 0 - [1] Liverpool'

tracking_home = data_attack.loc[play]
tracking_away = data_defence.loc[play]

tracking_home.columns = tracking_home.columns.str.replace('attack','Home')
tracking_away.columns = tracking_away.columns.str.replace('defense','Away')
tracking_home.shape, tracking_away.shape

((165, 114), (165, 109))

In [26]:
pd.DataFrame([[play, 
               data_attack.loc[play,'score'][0],
               data_attack.loc[play].index[0], 
               data_attack.loc[play].index[-1]] for play in data_attack.index.get_level_values(0).unique()], columns=['PLAY','SCORE','START','END']).to_dict(orient='records')

[{'PLAY': 'Bayern 0 - [1] Liverpool',
  'SCORE': 'BAY 0 [1] LIV',
  'START': 0,
  'END': 164},
 {'PLAY': 'Bournemouth 0 - 3 Liverpool',
  'SCORE': 'BOU 0 [3] LIV',
  'START': 0,
  'END': 170},
 {'PLAY': 'Fulham 0 - [1] Liverpool',
  'SCORE': 'FUL 0 [1] LIV',
  'START': 0,
  'END': 182},
 {'PLAY': 'Genk 0 - [3] Liverpool',
  'SCORE': 'GNK 0 [3] LIV',
  'START': 0,
  'END': 182},
 {'PLAY': 'Leicester 0 - [3] Liverpool',
  'SCORE': 'LEI 0 [3] LIV',
  'START': 0,
  'END': 124},
 {'PLAY': 'Liverpool [1] - 0 Everton',
  'SCORE': 'LIV [1] 0 EVE',
  'START': 0,
  'END': 198},
 {'PLAY': 'Liverpool [1] - 0 Watford',
  'SCORE': 'LIV [1] 0 WAT',
  'START': 0,
  'END': 224},
 {'PLAY': 'Liverpool [1] - 0 Wolves',
  'SCORE': 'LIV [1] 0 WLV',
  'START': 0,
  'END': 156},
 {'PLAY': 'Liverpool [2] - 0 Everton',
  'SCORE': 'LIV [2] 0 EVE',
  'START': 0,
  'END': 286},
 {'PLAY': 'Liverpool [2] - 0 Man City',
  'SCORE': 'LIV [2] 0 MCI',
  'START': 0,
  'END': 166},
 {'PLAY': 'Liverpool [2] - 0 Porto',
  'S

In [19]:
for play in data_attack.index.get_level_values(0).unique():
    print ([play, data_attack.loc[play,'score'][0],data_attack.loc[play].index[0], data_attack.loc[play].index[-1]])

Bayern 0 - [1] Liverpool BAY 0 [1] LIV 0 164
Bournemouth 0 - 3 Liverpool BOU 0 [3] LIV 0 170
Fulham 0 - [1] Liverpool FUL 0 [1] LIV 0 182
Genk 0 - [3] Liverpool GNK 0 [3] LIV 0 182
Leicester 0 - [3] Liverpool LEI 0 [3] LIV 0 124
Liverpool [1] - 0 Everton LIV [1] 0 EVE 0 198
Liverpool [1] - 0 Watford LIV [1] 0 WAT 0 224
Liverpool [1] - 0 Wolves LIV [1] 0 WLV 0 156
Liverpool [2] - 0 Everton LIV [2] 0 EVE 0 286
Liverpool [2] - 0 Man City LIV [2] 0 MCI 0 166
Liverpool [2] - 0 Porto LIV [2] 0 POR 0 194
Liverpool [2] - 0 Salzburg LIV [2] 0 RBS 0 190
Liverpool [2] - 1 Chelsea LIV [2] 1 CHE 0 194
Liverpool [2] - 1 Newcastle LIV [2] 1 NEW 0 148
Liverpool [3] - 0 Bournemouth LIV [3] 0 BOU 0 154
Liverpool [3] - 0 Norwich LIV [3] 0 NOR 0 148
Liverpool [4] - 0 Barcelona LIV [4] 0 FCB 0 138
Porto 0 - [2] Liverpool POR 0 [2] LIV 0 256
Southampton 1 - [2] Liverpool SOU 1 [2] LIV 0 256


# Params & Helpers

In [58]:
PLAYERMARKERSIZE = 20
player_marker_args = {'Home': dict(mode='markers+text', 
                                   marker_size = PLAYERMARKERSIZE, 
                                   marker_line_color="white", 
                                   marker_color="red", 
                                   marker_line_width=2,
                                   textfont=dict(
                                        size=11,
                                        color="white"
                                    )),
                      'Away': dict(mode='markers+text', 
                                   marker_size = PLAYERMARKERSIZE, 
                                   marker_line_color="white", 
                                   marker_color="#0570b0", 
                                   marker_line_width=2,
                                   textfont=dict(
                                        size=11,
                                        color="white"
                                    ))}

event_player_marker_args = {'Home': dict(mode='lines+markers+text', 
                                   marker_size = PLAYERMARKERSIZE, 
                                   marker_line_color="white", 
                                   marker_color="red", 
                                   marker_line_width=2,
                                   line_color = "red",
                                   textfont=dict(
                                        size=11,
                                        color="white"
                                    )),
                      'Away': dict(mode='lines+markers+text', 
                                   marker_size = PLAYERMARKERSIZE, 
                                   marker_line_color="white", 
                                   marker_color="#0570b0", 
                                   marker_line_width=2,
                                   line_color ="#0570b0",
                                   textfont=dict(
                                        size=11,
                                        color="white"
                                    ))}

In [59]:
from collections import namedtuple
point = namedtuple('point',['x','y'])
centre = point(0.,0)

In [60]:
# get pitch layout
def generate_pitch_layout(tracking_frames=None, field_dimen = (106.0,68.0), field_color ='green', linewidth=2, markersize=20):
    # decide what color we want the field to be. Default is green, but can also choose white
#     if field_color=='green':
#         ax.set_facecolor('mediumseagreen')
#         lc = 'whitesmoke' # line color
#         pc = 'w' # 'spot' colors
#     elif field_color=='white':
#         lc = 'k'
#         pc = 'k'
    # ALL DIMENSIONS IN m
    border_dimen = (3,3) # include a border arround of the field of width 3m
    meters_per_yard = 0.9144 # unit conversion from yards to meters
    half_pitch_length = field_dimen[0]/2. # length of half pitch
    half_pitch_width = field_dimen[1]/2. # width of half pitch
    signs = [-1,1] 
    # Soccer field dimensions typically defined in yards, so we need to convert to meters
    goal_line_width = 8*meters_per_yard
    box_width = 20*meters_per_yard
    box_length = 6*meters_per_yard
    area_width = 44*meters_per_yard
    area_length = 18*meters_per_yard
    penalty_spot = 12*meters_per_yard
    corner_radius = 1*meters_per_yard
    D_length = 8*meters_per_yard
    D_radius = 10*meters_per_yard
    D_pos = 12*meters_per_yard
    centre_circle_radius = 10*meters_per_yard
    centre = point(0.,0.)
    
    shapes = []
    
    mid_circle = dict(type="circle", x0=centre.x - centre_circle_radius, y0=centre.y - centre_circle_radius, x1=centre.x + centre_circle_radius, y1=centre.y + centre_circle_radius, line_color="White", layer = "below") 
    mid_line = dict(type="line",
                    x0=centre.x, y0=centre.y - half_pitch_width,
                    x1=centre.x, y1=centre.y + half_pitch_width, line_color="White", layer = "below")
    mid_point = dict(type="circle", 
                     x0=centre.x-0.4, y0=centre.y-0.4, 
                     x1=centre.x+0.4, y1=centre.y+0.4, line_color="White", fillcolor='white', layer = "below") 
    shapes.extend([mid_circle,mid_line,mid_point])
    
    
    for s in signs:
        
        # plot pitch boundary
        boundary1 = dict(type="line",
                    x0=-half_pitch_length, y0=s*half_pitch_width,
                    x1=half_pitch_length, y1=s*half_pitch_width, line_color="White", layer = "below")
        
        boundary2 = dict(type="line",
                    x0=s*half_pitch_length, y0=-half_pitch_width,
                    x1=s*half_pitch_length, y1=half_pitch_width, line_color="White", layer = "below")
        
        circle = dict(type="circle", 
                           x0=s*(centre.x+half_pitch_length-penalty_spot) - centre_circle_radius, y0=centre.y - centre_circle_radius,
                           x1=s*(centre.x+half_pitch_length-penalty_spot) + centre_circle_radius, y1=centre.y + centre_circle_radius, line_color="White", layer = "below")
        
        patch = dict(type="rect",
                     x0=s*half_pitch_length,y0=-area_width/2. - 1,
                     x1=s*(half_pitch_length - area_length),y1=area_width/2. + 1,line=dict(color="mediumseagreen",width=0),fillcolor="mediumseagreen", layer = "below")
        
        box = dict(type="rect",
                              x0=s*half_pitch_length,y0=-area_width/2.,
                              x1=s*(half_pitch_length - area_length),y1=area_width/2.,line=dict(color="white",width=2), layer = "below")
        D = dict(type="rect",
                 x0=s*half_pitch_length,y0=-box_width/2.,
                 x1=s*(half_pitch_length - box_length),y1=box_width/2.,line=dict(color="white",width=2), layer = "below")
        pen = dict(type="circle", 
                   x0=s*(half_pitch_length - penalty_spot)-0.4, y0=-0.4, 
                   x1=s*(half_pitch_length - penalty_spot)+0.4, y1= 0.4, line_color="White", fillcolor='white', layer = "below")

        top_post = dict(type="rect", 
                        x0=s*half_pitch_length, y0=goal_line_width/2. - 0.5, 
                        x1=s*(half_pitch_length-0.5), y1=goal_line_width/2.+0.5, line=dict(color="white",width=0),fillcolor="white", layer = "below")
        
        bottom_post = dict(type="rect", 
                        x0=s*half_pitch_length, y0=-goal_line_width/2. - 0.5, 
                        x1=s*(half_pitch_length-0.5), y1=-goal_line_width/2.+0.5, line=dict(color="white",width=0),fillcolor="white", layer = "below")
        
        shapes.extend([boundary1, boundary2,circle, patch , box, D, pen, top_post, bottom_post])
        
        
    
    sliders_dict = {
    "active": 0,
#     "yanchor": "top",
#     "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Frame:",
        "visible": True,
        "xanchor": "right"
    },
#     "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
#     "len": 0.9,
#     "x": 0.1,
#     "y": 0,
    "steps": []
    }
    
    for frame_num in tracking_frames:        
        slider_step = {"args": [
        [frame_num],
        {"frame": {"duration": 0, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 0}
        }
        ],
        "label": frame_num,
        "method": "animate"
                      }
        sliders_dict["steps"].append(slider_step)
        
    updatemenus = {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 0, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 0,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
    
    
    # set axis limits
    xmax = field_dimen[0]/2. + border_dimen[0]
    ymax = field_dimen[1]/2. + border_dimen[1]
    
    layout = go.Layout(
                    title='Plotting frames and events using Plotly',
                    hovermode='closest',
                    autosize=False,
                    width=1200,
                    height=800,
                    plot_bgcolor='mediumseagreen',
                    xaxis=go.layout.XAxis(range=[-xmax, xmax],
                                          showgrid=False,zeroline=False,
                                          showticklabels=False
                                          ),
                    yaxis=go.layout.YAxis(range=[-ymax, ymax],
                                          showgrid=False,zeroline=False,
                                          showticklabels=False,
                                          scaleanchor = "x",
                                          scaleratio = 1,
                    ))
    
    layout["shapes"] = shapes
    layout["sliders"] = [sliders_dict] 
    layout["updatemenus"] = [updatemenus] 
    
    
    return layout

In [69]:
def positions(tracking_data):
    position_traces = []
    for i, side in enumerate(['Home','Away']):
        team_data = tracking_data[i]
        player_nums = list(set(item for subitem in team_data.keys() for item in subitem.split('_')
                               if item.isdigit()))
        xlocs = [team_data['{}_{}_{}'.format(side, num,'x')] for num in player_nums]
        ylocs = [team_data['{}_{}_{}'.format(side, num,'y')] for num in player_nums]
        traces = go.Scatter(x = xlocs,y = ylocs, text = player_nums, **player_marker_args[side], name = side)
        position_traces.append(traces)

    return position_traces


def velocities(tracking_data):
    velocity_quivers = []
    for i, side in enumerate(['Home','Away']):
        team_data = tracking_data[i]
        player_nums = list(set(item for subitem in team_data.keys() for item in subitem.split('_')
                               if item.isdigit()))
        xlocs = [team_data['{}_{}_{}'.format(side, num,'x')] for num in player_nums]
        ylocs = [team_data['{}_{}_{}'.format(side, num,'y')] for num in player_nums]
        xvels = [team_data['{}_{}_{}'.format(side, num,'vx')] for num in player_nums]
        yvels = [team_data['{}_{}_{}'.format(side, num,'vy')] for num in player_nums]
        trace = ff.create_quiver(x = xlocs, y = ylocs, u = xvels,v = yvels, 
                                 scale=.5, line_color = player_marker_args[side]['marker_color'], name = side+'_vel')
        velocity_quivers.append(trace.data[0])

    return velocity_quivers



def plot_ball(tracking_data):
    team_data = tracking_data[0]
    trace = go.Scatter(x = [team_data['ball_x']],y = [team_data['ball_y']], 
                           marker_size = 10, marker_opacity=0.8, marker_color = '#262e39', name = 'ball'
                          )
    return [trace]

def generate_data_for_frame(frame_num, show_velocities=True):
    hometeam = tracking_home.loc[frame_num]
    awayteam = tracking_away.loc[frame_num]
    tracking_data = (hometeam,awayteam)
    traces = []
    
    if show_velocities: traces.extend(velocities(tracking_data))
        
    traces.extend(positions(tracking_data))
    
    if hometeam['ball_x']: traces.extend(plot_ball(tracking_data))
    
    data = traces
    
    return data


def generate_plotly_frames_for_event(tracking_frames):
    frames = []
    for frame_num in tracking_frames:#[::25]:
        frame = {"data": [], "name": str(frame_num)}
        frame["data"].extend(generate_data_for_frame(frame_num = frame_num))
#         print (len(frame['data']))
        frames.append(frame)
    
    return frames
    

In [70]:
def plot_pitch_slider(event_start_frame, event_end_frame,field_dimen = (106.0,68.0), field_color ='green', linewidth=2, markersize=20, data = None, frame= None):
    tracking_frames = range(event_start_frame, event_end_frame+1)
    # make figure
    fig_dict = {
        "data": [],
        "layout": {},
        "frames": []
    }
    fig_dict['layout'] = generate_pitch_layout(tracking_frames, field_dimen = field_dimen, field_color =field_color, linewidth=linewidth, markersize=markersize)
    fig_dict['data'] = generate_data_for_frame(frame_num = event_start_frame)
    fig_dict['frames'] = generate_plotly_frames_for_event(tracking_frames)
    
#     with open('../output/fig_dict.json','w') as fp:
#         json.dump(fig_dict, fp)
#     print ('dumped')
    fig = go.Figure(fig_dict)

    fig.show()

# Plot

In [4]:
plot_pitch_slider() 
# fig = go.Figure(fig_dict)
# fig.show()

func:'generate_pitch_layout' took: 0.2071 sec


100%|██████████| 287/287 [06:43<00:00,  1.41s/it]  


func:'generate_plotly_frames_for_event' took: 403.6546 sec
func:'plot_pitch_slider' took: 412.6940 sec


# Pitch Control

## for a frame

In [ ]:
play = 'Bayern 0 - [1] Liverpool'

In [8]:
hometeam = data_attack.loc[play]#.head(3)#.columns.tolist()
awayteam = data_defence.loc[play]#.head(3)#.columns.tolist()

hometeam.columns = hometeam.columns.str.replace('attack', 'Home')
awayteam.columns = awayteam.columns.str.replace('defense', 'Away')    
    
# first get model parameters
params = mpc.default_model_params(3)
frame = 160
attacking='Home'
field_dimen=(106.0, 68.0)
n_grid_cells_x=50
# get the details of the frame: team in possession, ball_start_position)
ball_start_pos = np.array(hometeam.loc[frame,['ball_x', 'ball_y']].tolist())


In [11]:
hometeam.loc[frame,['ball_x', 'ball_y']].tolist()
hometeam.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            155, 156, 157, 158, 159, 160, 161, 162, 163, 164],
           dtype='int64', name='frame', length=165)

In [ ]:

pitch_control_xy_df = pd.DataFrame()
for frame in tqdm(hometeam.index):
    # get the details of the frame: team in possession, ball_start_position)
    ball_start_pos = np.array(hometeam.loc[frame,['ball_x', 'ball_y']].tolist())
    
    # ball_start_pos = np.array([g1.loc[event_id]['Start X'],g1.loc[event_id]['Start Y']])
    # print (np.linalg.norm( target_position - ball_start_pos )/params['average_ball_speed'])
    
    # initialise player positions and velocities for pitch control calc (so that we're not repeating this at each grid cell position)
    attacking_players = mpc.initialise_players(hometeam.loc[frame],'Home',params)
    defending_players = mpc.initialise_players(awayteam.loc[frame],'Away',params)
    opp='Away'

    

    PPCFatt, PPCFdef, Patt, Pdef= mpc.calculate_pitch_control_at_target(target_position, 
                                      attacking_players, defending_players, 
                                      ball_start_pos, params, 
                                      return_individual=True)
    
    pitch_control_xy_df.loc[frame,'PPCFatt'] = PPCFatt
    pitch_control_xy_df.loc[frame,'PPCFdef'] = PPCFdef
    pitch_control_xy_df.loc[frame,'attacking'] = team
    for pid, pc in Patt.items():
        pitch_control_xy_df.loc[frame,"%s_%s_%s" % (team,pid, 'pc')] = pc
    for pid, pc in Pdef.items():
        pitch_control_xy_df.loc[frame,"%s_%s_%s" % (opp,pid, 'pc')] = pc

In [12]:
mpc.generate_pitch_control_for_frame(hometeam, awayteam, 160, params, attacking='Home',return_individual=True)

(array([[0.03725649, 0.10251751, 0.25788864, ..., 0.9699992 , 0.97063826,
         0.9711461 ],
        [0.01746784, 0.05331291, 0.15706866, ..., 0.97126215, 0.96985467,
         0.97019246],
        [0.0075482 , 0.02446061, 0.08328677, ..., 0.97007614, 0.97026156,
         0.9703956 ],
        ...,
        [0.75322127, 0.72757729, 0.69728244, ..., 0.96607756, 0.96533699,
         0.96449124],
        [0.75277186, 0.72999157, 0.70607513, ..., 0.96657306, 0.96608177,
         0.96546441],
        [0.75278857, 0.73467357, 0.72127622, ..., 0.96850496, 0.9682321 ,
         0.96783062]]),
 array([-53.        , -50.83673469, -48.67346939, -46.51020408,
        -44.34693878, -42.18367347, -40.02040816, -37.85714286,
        -35.69387755, -33.53061224, -31.36734694, -29.20408163,
        -27.04081633, -24.87755102, -22.71428571, -20.55102041,
        -18.3877551 , -16.2244898 , -14.06122449, -11.89795918,
         -9.73469388,  -7.57142857,  -5.40816327,  -3.24489796,
         -1.08163265,   1

In [23]:
g1.tail()

,attack_10_x,attack_11_x,attack_12_x,attack_13_x,attack_14_x,attack_15_x,attack_16_x,attack_17_x,attack_18_x,attack_19_x,attack_1_x,attack_20_x,attack_21_x,attack_22_x,attack_2_x,attack_3_x,attack_4_x,attack_5_x,attack_6_x,attack_7_x,attack_8_x,attack_9_x,ball_x,attack_10_y,attack_11_y,attack_12_y,attack_13_y,attack_14_y,attack_15_y,attack_16_y,attack_17_y,attack_18_y,attack_19_y,attack_1_y,attack_20_y,attack_21_y,attack_22_y,attack_2_y,attack_3_y,attack_4_y,attack_5_y,attack_6_y,attack_7_y,attack_8_y,attack_9_y,ball_y,Time [s],attack_1_vx,attack_1_vy,attack_1_speed,attack_10_vx,attack_10_vy,attack_10_speed,attack_11_vx,attack_11_vy,attack_11_speed,attack_12_vx,attack_12_vy,attack_12_speed,attack_13_vx,attack_13_vy,attack_13_speed,attack_14_vx,attack_14_vy,attack_14_speed,attack_15_vx,attack_15_vy,attack_15_speed,attack_16_vx,attack_16_vy,attack_16_speed,attack_17_vx,attack_17_vy,attack_17_speed,attack_18_vx,attack_18_vy,attack_18_speed,attack_19_vx,attack_19_vy,attack_19_speed,attack_2_vx,attack_2_vy,attack_2_speed,attack_20_vx,attack_20_vy,attack_20_speed,attack_21_vx,attack_21_vy,attack_21_speed,attack_22_vx,attack_22_vy,attack_22_speed,attack_3_vx,attack_3_vy,attack_3_speed,attack_4_vx,attack_4_vy,attack_4_speed,attack_5_vx,attack_5_vy,attack_5_speed,attack_6_vx,attack_6_vy,attack_6_speed,attack_7_vx,attack_7_vy,attack_7_speed,attack_8_vx,attack_8_vy,attack_8_speed,attack_9_vx,attack_9_vy,attack_9_speed,score
frame,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
160,-16.542410,-14.435069,NaN,NaN,NaN,NaN,NaN,NaN,-22.124487,-2.477931,-34.360034,-49.729452,-24.939202,NaN,-2.055615,NaN,NaN,NaN,NaN,NaN,-35.281090,-16.287176,-53.410769,18.629390,-6.352042,NaN,NaN,NaN,NaN,NaN,NaN,-17.627357,2.483625,-14.603285,2.833205,-23.260655,NaN,-13.768813,NaN,NaN,NaN,NaN,NaN,-7.677699,4.650714,2.806203,8.00,4.914993,-0.913498,4.999163,-3.845330,-2.176047,4.418342,-3.093384,0.163140,3.097683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.177069,1.162906,4.335926,-1.651917,-0.245053,1.669994,-0.733903,0.323077,0.801868,-7.113110,0.329824,7.120753,-4.458031,2.071474,4.915796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.471145,1.037182,3.622788,-1.637147,-0.828230,1.834725,BAY 0 [1] LIV
161,-16.766777,-14.615948,NaN,NaN,NaN,NaN,NaN,NaN,-22.366027,-2.564008,-34.069510,-50.154187,-25.200142,NaN,-2.079986,NaN,NaN,NaN,NaN,NaN,-35.486395,-16.382495,-53.575803,18.502036,-6.342137,NaN,NaN,NaN,NaN,NaN,NaN,-17.561163,2.477921,-14.659565,2.882284,-23.139428,NaN,-13.743776,NaN,NaN,NaN,NaN,NaN,-7.622512,4.602925,2.985242,8.05,4.116117,-0.774436,4.188337,-3.204705,-1.815302,3.683131,-2.579808,0.137741,2.583483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.471101,0.961511,3.601811,-1.328256,-0.163906,1.338331,-0.525070,0.298246,0.603862,-5.973421,0.414549,5.987789,-3.719125,1.728032,4.100973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.905635,0.839800,3.024563,-1.363454,-0.687775,1.527102,BAY 0 [1] LIV
162,-16.991189,-14.797159,NaN,NaN,NaN,NaN,NaN,NaN,-22.605928,-2.642789,-33.777921,-50.586056,-25.461768,NaN,-2.091156,NaN,NaN,NaN,NaN,NaN,-35.693878,-16.477675,-53.721088,18.374358,-6.331933,NaN,NaN,NaN,NaN,NaN,NaN,-17.496238,2.478368,-14.718585,2.952931,-23.017900,NaN,-13.714286,NaN,NaN,NaN,NaN,NaN,-7.571429,4.555540,3.142857,8.10,3.041838,-1.167581,3.258224,-2.506991,-1.054108,2.719586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.266217,-0.255469,0.368966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.128392,-0.037517,1.129015,BAY 0 [1] LIV
163,-16.991189,-14.797159,NaN,NaN,NaN,NaN,NaN,NaN,-22.605928,-2.642789,-33.777921,-50.586056,-25.461768,NaN,-2.091156,NaN,NaN,NaN,NaN,NaN,-35.693878,-16.477675,-53.721088,18.374358,-6.331933,NaN,NaN,NaN,NaN,NaN,NaN

In [28]:
target_position = np.array([g1.loc[event_id,['End X']],g1.loc[event_id,['End Y']]])
team = g1.loc[event_id].Team

pass_start_frame = g1.loc[event_id]['Start Frame']
pass_end_frame = g1.loc[event_id]['End Frame']
n=500
# lookup n frames prior to pass end frame
hometeam = tracking_home.loc[pass_end_frame-n:pass_end_frame]
awayteam = tracking_away.loc[pass_end_frame-n:pass_end_frame]

pitch_control_xy_df = pd.DataFrame()
for frame in tqdm(range(pass_end_frame-n,pass_end_frame)):
    # get the details of the frame: team in possession, ball_start_position)
    ball_start_pos = np.array(hometeam.loc[frame,['ball_x', 'ball_y']])
    # ball_start_pos = np.array([g1.loc[event_id]['Start X'],g1.loc[event_id]['Start Y']])
# print (np.linalg.norm( target_position - ball_start_pos )/params['average_ball_speed'])
    # initialise player positions and velocities for pitch control calc (so that we're not repeating this at each grid cell position)
    if team=='Home':
        attacking_players = mpc.initialise_players(hometeam.loc[frame],'Home',params)
        defending_players = mpc.initialise_players(awayteam.loc[frame],'Away',params)
        opp='Away'
    elif team=='Away':
        defending_players = mpc.initialise_players(hometeam.loc[frame],'Home',params)
        attacking_players = mpc.initialise_players(awayteam.loc[frame],'Away',params)
        opp='Home'
    else:
        assert False, "Team in possession must be either home or away"
    

    PPCFatt, PPCFdef, Patt, Pdef= mpc.calculate_pitch_control_at_target(target_position, 
                                      attacking_players, defending_players, 
                                      ball_start_pos, params, 
                                      return_individual=True)
    
    pitch_control_xy_df.loc[frame,'PPCFatt'] = PPCFatt
    pitch_control_xy_df.loc[frame,'PPCFdef'] = PPCFdef
    pitch_control_xy_df.loc[frame,'attacking'] = team
    for pid, pc in Patt.items():
        pitch_control_xy_df.loc[frame,"%s_%s_%s" % (team,pid, 'pc')] = pc
    for pid, pc in Pdef.items():
        pitch_control_xy_df.loc[frame,"%s_%s_%s" % (opp,pid, 'pc')] = pc

MultiIndex([(     'Bayern 0 - [1] Liverpool',   0),
            (     'Bayern 0 - [1] Liverpool',   1),
            (     'Bayern 0 - [1] Liverpool',   2),
            (     'Bayern 0 - [1] Liverpool',   3),
            (     'Bayern 0 - [1] Liverpool',   4),
            (     'Bayern 0 - [1] Liverpool',   5),
            (     'Bayern 0 - [1] Liverpool',   6),
            (     'Bayern 0 - [1] Liverpool',   7),
            (     'Bayern 0 - [1] Liverpool',   8),
            (     'Bayern 0 - [1] Liverpool',   9),
            ...
            ('Southampton 1 - [2] Liverpool', 247),
            ('Southampton 1 - [2] Liverpool', 248),
            ('Southampton 1 - [2] Liverpool', 249),
            ('Southampton 1 - [2] Liverpool', 250),
            ('Southampton 1 - [2] Liverpool', 251),
            ('Southampton 1 - [2] Liverpool', 252),
            ('Southampton 1 - [2] Liverpool', 253),
            ('Southampton 1 - [2] Liverpool', 254),
            ('Southampton 1 - [2] Liverpool', 25